In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from __future__ import division
import nltk
from wikipedia import page

In [30]:
data = pd.read_csv("user_question.csv", header=None)
data.columns = ['question','group']

In [31]:
train = data['question'].values.tolist()
label = data['group'].values.tolist()

In [32]:
corpus = train + label

In [35]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [36]:
tok_corp = [nltk.word_tokenize(sent.decode('utf-8')) for sent in corpus]

In [37]:
model = Word2Vec(tok_corp, size=50, window=5, min_count=3, workers=2)

In [45]:
model.most_similar('How')

[(u'curl', 0.2829551100730896),
 (u'want', 0.14266327023506165),
 (u'What', 0.13052792847156525),
 (u'to', 0.11642688512802124),
 (u'Shoulders', 0.08410151302814484),
 (u'abs', 0.05228054150938988),
 (u'Triceps', 0.01863226667046547),
 (u'I', 0.006797999143600464),
 (u'do', -0.019749730825424194),
 (u'Leg', -0.023615125566720963)]

In [51]:
wikipage = page("Physical fitness")